# Breakdown analysis of life cycle GHG emissions for LIB raw materials

In [1]:
%run _imports.ipynb

In [3]:
BW_PROJECT = 'lib_rm' # insert your project name here
bw.projects.set_current(BW_PROJECT)

EI_DB = 'ecoinvent-3.10-cutoff' # name of ecoinvent database in your project
LIB_RM_DB = "LIB raw materials"

IMPACT_METHODS = {"Climate change": ('IPCC 2021', 'climate change: including SLCFs', 'global warming potential (GWP100)')}

# (name, reference product, location)
INVENTORIES = {
    "Lithium, brine":        ("lithium hydroxide production, Salar de Atacama", "lithium hydroxide, battery grade", "CL"),
    "Lithium, spodumene":    ("lithium hydroxide production, from Australian spodumene", "lithium hydroxide, battery grade", "CN"),
    "Cobalt":                ("cobalt sulfate production, from copper-cobalt ore, economic allocation", "cobalt sulfate", "CN"),
    "Nickel":                ("nickel sulfate production, average excluding China, economic allocation", "nickel sulfate", "GLO"),
    "Graphite, natural":     ("natural graphite production, battery grade, from Heilongjiang", "natural graphite, battery grade", "CN"),
    "Graphite, synthetic":   ("graphite powder coating", "synthetic graphite, battery grade", "CN"),
    
    "Lithium, brine - carbonate":        ("lithium carbonate production, Salar de Atacama", "lithium carbonate, battery grade", "CL"),
    "Lithium, spodumene - carbonate":    ("lithium carbonate production, from Australian spodumene", "lithium carbonate, battery grade", "CN"),
}

INVENTORIES_ds = {}
for rm in INVENTORIES:
    rm_ds = [ds for ds in bw.Database(LIB_RM_DB) if ds["name"] == INVENTORIES[rm][0] 
                                                 and ds["reference product"]==INVENTORIES[rm][1] 
                                                 and ds["location"]==INVENTORIES[rm][2]][0]
    INVENTORIES_ds.update({rm: rm_ds})

# Initialize LCA object
lca = supporting_functions.init_simple_lca(INVENTORIES_ds['Lithium, brine'])

### Life cycle GHG emissions

In [4]:
total_ghg_emissions = {}
for rm in INVENTORIES_ds:
    impacts = supporting_functions.multi_lcia(lca, INVENTORIES_ds[rm], IMPACT_METHODS)
    total_ghg_emissions[rm] = impacts

total_ghg_emissions = pd.DataFrame(total_ghg_emissions).T

In [5]:
# Export total GHG emissions
total_ghg_emissions.to_csv(DATA_DIR / "results" / f"fig2_total_ghgs_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Raw material")

### GHG emissions breakdown analysis

In [6]:
CONTRIBUTORS_LIST = [
    "Electricity consumption",
    "Process heating",
    "Fuels consumption",
    "Reagents consumption",
    "Process emissions",
    "Other"
    ]

breakdown_lists = supporting_functions.get_breakdown_lists()

In [7]:
inventories_breakdown = pd.read_excel(INVENTORIES_PATH, sheet_name="Datasets for breakdown", index_col=0, skiprows=1)
inventories_breakdown

,name,reference product,location,amount,stage
raw material,,,,,
"Lithium, brine","lithium hydroxide production, Salar de Atacama","lithium hydroxide, battery grade",CL,1.000000,Refining
"Lithium, brine","lithium carbonate production, Salar de Atacama","lithium carbonate, battery grade",CL,1.050000,Refining
"Lithium, brine","lithium brine purification, Salar de Atacama",purified lithium brine,CL,22.884815,Concentration
"Lithium, brine","lithium brine production, from evaporation pon...","lithium brine, from evaporation pond",CL,3.862008,Mining
"Lithium, spodumene","lithium hydroxide production, from Australian ...","lithium hydroxide, battery grade",CN,1.000000,Refining
"Lithium, spodumene",spodumene concentrate production,spodumene concentrate,AU,6.420000,Concentration
Cobalt,"cobalt sulfate production, from copper-cobalt ...",cobalt sulfate,CN,1.000000,Refining
Cobalt,"cobalt hydroxide, hydrometallurigcal procesing...",cobalt hydroxide,CD,1.100000,Concentration
Cobalt,"copper-cobalt mining, industrial",copper-cobalt ore,CD,50.600000,Mining


In [33]:
impacts_breakdown = {}

for rm in list(set(inventories_breakdown.index)):
    print("##########################")
    print(rm)
    print("##########################")
    
    impacts_breakdown[rm] = {}
    skip_inventories = list(inventories_breakdown.loc[rm]["name"])

    for index, row in inventories_breakdown.loc[rm].iterrows():
        inv_amount = row["amount"]
        try:
            inv = [ds for ds in bw.Database(LIB_RM_DB)
                if ds['name'] == row["name"]
                and ds['reference product'] == row["reference product"]
                and ds['location'] == row["location"]][0]
        except IndexError:
            inv = [ds for ds in bw.Database(EI_DB)
                if ds['name'] == row["name"]
                and ds['reference product'] == row["reference product"]
                and ds['location'] == row["location"]][0]
         
        print(inv_amount, "|", inv)
        
        impacts_breakdown[rm][row["name"]] = supporting_functions.lcia_system_contribution(lca, inv, skip_inventories, 
                IMPACT_METHODS, CONTRIBUTORS_LIST, breakdown_lists, activity_amount=inv_amount)

##########################
Graphite, natural
##########################
1.0 | 'natural graphite production, battery grade, from Heilongjiang' (kilogram, CN, None)
1.01 | 'natural graphite purification' (kilogram, CN, None)
1.1413 | 'natural graphite spheronization' (kilogram, CN, None)
2.5336860000000003 | 'natural graphite concentration' (kilogram, CN, None)
24.298048740000002 | 'graphite ore mining' (kilogram, CN, None)
##########################
Lithium, spodumene - carbonate
##########################
1.0 | 'lithium carbonate production, from Australian spodumene' (kilogram, CN, None)
7.3 | 'spodumene concentrate production' (kilogram, AU, None)
##########################
Graphite, synthetic
##########################
1.0 | 'graphite powder coating' (kilogram, CN, None)
0.97 | 'micronization of graphite powder' (kilogram, CN, None)
1.6166666666666667 | 'graphitization of calcined coke powder' (kilogram, CN, None)
1.6199064796259184 | 'calcined coke milling' (kilogram, CN, None)
1.5

In [34]:
ghgs_breakdown = {outer_key: {
    inner_key: sub_dict["Climate change"] 
    for inner_key, sub_dict in inner_dict.items()} 
    for outer_key, inner_dict in impacts_breakdown.items()
}

In [35]:
ghgs_breakdown_stages = {
    metal: { 
        stage: {
            contributor: 0 
            for contributor in CONTRIBUTORS_LIST
        } 
        for stage in (
            ["Coating", "Micronization", "Graphitization", "Milling", "Graphite crucibles", "Calcination"]
            if metal == "Graphite, synthetic" else
            ["Mining", "Concentration", "Refining"]
        )
    } 
    for metal in INVENTORIES.keys()
}

for rm in ghgs_breakdown:
    total = 0
    for up in ghgs_breakdown[rm]:
        stage = inventories_breakdown.loc[rm][inventories_breakdown.loc[rm]["name"] == up]["stage"][0]
        for contributor in ghgs_breakdown[rm][up]:
            ghgs_breakdown_stages[rm][stage][contributor] += ghgs_breakdown[rm][up][contributor]

C:\Users\istrateir\AppData\Local\Temp\ipykernel_23064\85110419.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stage = inventories_breakdown.loc[rm][inventories_breakdown.loc[rm]["name"] == up]["stage"][0]


In [36]:
ghgs_breakdown_stages_df = pd.DataFrame([(rm, stage, source, value)
                        for rm, stage in ghgs_breakdown_stages.items()
                        for stage, source in stage.items()
                        for source, value in source.items()],
                        columns=['Raw material', 'Stage', 'Source', 'Value']).set_index("Metal")

In [38]:
# Export breakdown results
ghgs_breakdown_stages_df.to_csv(DATA_DIR / "results" / f"fig2_breakdown_ghgs_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Raw material")

In [39]:
# Sanity check, breakdown sum vs total emissions
for metal in ghgs_breakdown_stages:
    totals = 0
    for stage in ghgs_breakdown_stages[metal]:
        for contributor in ghgs_breakdown_stages[metal][stage]:
            totals += ghgs_breakdown_stages[metal][stage][contributor]
    print(metal, totals, "|", total_ghg_emissions.loc[metal].values[0], "| Diff:", totals - total_ghg_emissions.loc[metal].values[0])

Lithium, brine 7.7021847347669405 | 7.7021844210510055 | Diff: 3.1371593500750805e-07
Lithium, spodumene 17.237322659056534 | 17.237322941111596 | Diff: -2.820550619730966e-07
Cobalt 14.990897345381024 | 14.990897224184717 | Diff: 1.2119630632412282e-07
Nickel 5.479065089074897 | 5.479064852557049 | Diff: 2.3651784797351638e-07
Graphite, natural 10.560489081363658 | 10.560489127887507 | Diff: -4.652384966163936e-08
Graphite, synthetic 43.87147739733364 | 43.87147798797092 | Diff: -5.906372777531033e-07
Lithium, brine - carbonate 3.620094451905943 | 3.6200944006547635 | Diff: 5.125117930049328e-08
Lithium, spodumene - carbonate 24.105142094023122 | 24.105141974651914 | Diff: 1.1937120802940626e-07


### Breakdown GHG emissions from reagents consumption

In [9]:
reagents_impacts_breakdown = {}

for rm in list(set(inventories_breakdown.index)):
    print(rm)
    print("------------------------")
    reagents_impacts_breakdown[rm] = {}
    skip_inventories = list(inventories_breakdown.loc[rm]["name"])

    for index, row in inventories_breakdown.loc[rm].iterrows():
        inv_amount = row["amount"]
        try:
            inv = [ds for ds in bw.Database(LIB_RM_DB)
                if ds['name'] == row["name"]
                and ds['reference product'] == row["reference product"]
                and ds['location'] == row["location"]][0]
        except IndexError:
            inv = [ds for ds in bw.Database(EI_DB)
                if ds['name'] == row["name"]
                and ds['reference product'] == row["reference product"]
                and ds['location'] == row["location"]][0]

        reagents_impacts_breakdown[rm][row["name"]] = supporting_functions.lcia_reagents_disaggregation(
            lca, inv, skip_inventories, IMPACT_METHODS, breakdown_lists, activity_amount=inv_amount)

Lithium, spodumene - carbonate
------------------------
Cobalt
------------------------
Lithium, spodumene
------------------------
Lithium, brine - carbonate
------------------------
Graphite, natural
------------------------
Lithium, brine
------------------------
Graphite, synthetic
------------------------
Nickel
------------------------


In [10]:
reagents_ghgs_breakdown = {outer_key: {
    inner_key: sub_dict["Climate change"] 
    for inner_key, sub_dict in inner_dict.items()} 
    for outer_key, inner_dict in reagents_impacts_breakdown.items()
}

In [12]:
reagents_ghgs_breakdown_stages = {
        rm: { 
            stage: {
                contributor: 0 
                for contributor in breakdown_lists["reagent products"]} 
                for stage in (
            ["Coating", "Micronization", "Graphitization", "Milling", "Graphite crucibles", "Calcination"]
            if rm == "Graphite, synthetic" else
            ["Mining", "Concentration", "Refining"]
        )}
            for rm in list(set(inventories_breakdown.index))
        }

for rm in reagents_ghgs_breakdown:
    total = 0
    for up in reagents_ghgs_breakdown[rm]:
        stage = inventories_breakdown.loc[rm][inventories_breakdown.loc[rm]["name"] == up]["stage"][0]
        for contributor in reagents_ghgs_breakdown[rm][up]:
            reagents_ghgs_breakdown_stages[rm][stage][contributor] += reagents_ghgs_breakdown[rm][up][contributor]

C:\Users\istrateir\AppData\Local\Temp\ipykernel_34528\282864256.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stage = inventories_breakdown.loc[rm][inventories_breakdown.loc[rm]["name"] == up]["stage"][0]


In [13]:
reagents_ghgs_breakdown_stages_df = pd.DataFrame([(metal, stage, reagent, value)
                        for metal, stage in reagents_ghgs_breakdown_stages.items()
                        for stage, reagent in stage.items()
                        for reagent, value in reagent.items()],
                        columns=['Raw material', 'Stage', 'Reagent', 'Value']).set_index("Raw material")

In [14]:
# Export reagents breakdown results
reagents_ghgs_breakdown_stages_df.to_csv(DATA_DIR / "results" / f"ESI_breakdown_reagents_ghgs_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Metal")